In [2]:
from ultralytics import YOLO


In [3]:
# Load the model from local path
model = YOLO("best.pt")  # Ensure the file is in the same directory as your notebook

In [7]:
results = model("D:\Gun.jpg", save=True, show=True)



image 1/1 D:\Gun.jpg: 800x928 1 Gun, 3103.1ms
Speed: 334.7ms preprocess, 3103.1ms inference, 172.4ms postprocess per image at shape (1, 3, 800, 928)
Results saved to runs\detect\predict


In [8]:
results = model("D:\Explosive.jpg", save=True, show=True)



image 1/1 D:\Explosive.jpg: 544x928 1 Explosive_Objects, 515.9ms
Speed: 15.6ms preprocess, 515.9ms inference, 3.5ms postprocess per image at shape (1, 3, 544, 928)
Results saved to runs\detect\predict


In [10]:
results = model("D:\Knife.jpg", save=True, show=True)



image 1/1 D:\Knife.jpg: 736x928 1 knife, 543.1ms
Speed: 33.4ms preprocess, 543.1ms inference, 1.8ms postprocess per image at shape (1, 3, 736, 928)
Results saved to runs\detect\predict


In [11]:
!pip install pygame


In [12]:
import cv2
import torch
import threading
import os
import pygame 
from ultralytics import YOLO
from playsound import playsound

pygame 2.6.1 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [13]:
# Define alarm sound paths
ALARM_SOUNDS = {
    "Gun": "D:/Gun_Alarm.mp3",
    "Knife": "D:/Knife_Alarm.mp3",
    "Explosive_Object": "D:/Explosive_Alarm.mp3"  # Ensure the label exactly matches your model's output
}

def play_alarm(label):
    """Play the alarm sound in a separate thread to avoid blocking execution."""
    if label in ALARM_SOUNDS:
        threading.Thread(target=playsound, args=(ALARM_SOUNDS[label],), daemon=True).start()

def start_webcam():
    """Open webcam, detect objects using YOLO v11, and trigger alarms."""
    cap = cv2.VideoCapture(0)  # Open webcam

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame, exiting...")
            break
        
        # Run YOLO v11 Object Detection
        results = model(frame)

        detected_objects = []  # Store detected objects to avoid overwriting

        # Process detections
        for result in results:
            for box in result.boxes:
                cls = int(box.cls[0])  # Class index
                conf = float(box.conf[0])  # Confidence score
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box
                
                if conf > 0.5:
                    label = model.names[cls]  # Get class label
                    
                    # Append detected object for verification
                    detected_objects.append((label, conf, (x1, y1, x2, y2)))
        
        # Ensure correct alarms and labels are used
        for label, conf, (x1, y1, x2, y2) in detected_objects:
            # Assign colors: Red for High Alert, Blue for Mid Alert
            color = (255, 255, 0)  # Blue (Default)
            if label in ALARM_SOUNDS:
                color = (0, 0, 255)  # Red for High Alert
                print(f"⚠️ Alert: {label} detected! Confidence: {conf:.2f}")
                play_alarm(label)  # Play the correct alarm sound

            # Draw bounding box with correct label
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # Display frame with detections
        cv2.imshow("YOLOv11 Object Detection", frame)

        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord("q"):
            print("Webcam stopped by user.")
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
start_webcam()

In [14]:
pygame.mixer.init()

In [18]:

# Correct file paths with raw strings
ALARM_SOUNDS = {
    "explosive_objects": r"D:\Explosive_Alarm.mp3",
    "gun": r"D:\Gun_Alarm.mp3",
    "knife": r"D:\Knife_Alarm.mp3"
}

def play_alarm(label):
    """Play alarm sound based on detected object."""
    label = label.lower()  # Ensure case consistency
    if label in ALARM_SOUNDS and os.path.exists(ALARM_SOUNDS[label]):
        print(f"🚨 Playing alarm for {label}...")
        pygame.mixer.music.load(ALARM_SOUNDS[label])  # Load sound
        pygame.mixer.music.play()  # Play sound
        while pygame.mixer.music.get_busy():  # Wait until sound finishes playing
            continue
    else:
        print(f"⚠️ Alarm sound for {label} not found or file missing!")

def detect_objects(image_path):
    """Detect objects in the image and trigger alarms."""
    if not os.path.exists(image_path):
        print("❌ Error: Image file not found! Check the path.")
        return

    # Read image
    img = cv2.imread(image_path)

    # Run YOLO detection
    results = model(img)

    detected_labels = []  # Track detected objects

    for result in results:
        for box in result.boxes:
            cls = int(box.cls[0])  # Class index
            conf = float(box.conf[0])  # Confidence score
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box

            if conf > 0.5:
                label = model.names[cls].lower()  # Normalize label to lowercase
                detected_labels.append(label)  # Store detected label
                color = (0, 255, 0)  # Default: green (mid-alert)

                if label in ALARM_SOUNDS:
                    color = (0, 0, 255)  # Red for high-alert
                    print(f"⚠️ Alert: {label} detected! Confidence: {conf:.2f}")
                    print(f"🔊 Calling play_alarm('{label}')")

                    # Play alarm
                    play_alarm(label)

                # Draw bounding box
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                cv2.putText(img, f"{label} {conf:.2f}", (x1, y1 - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Show image
    cv2.imshow("YOLO Object Detection", img)
    cv2.waitKey(1)
    cv2.destroyAllWindows()

    # Final debugging message
    if detected_labels:
        print(f"✅ Objects detected: {', '.join(detected_labels)}")
    else:
        print("❌ No objects detected above confidence threshold.")

# Run detection (Replace with your image path)
image_path = r"D:\Knife.jpg"
detect_objects(image_path)



0: 736x928 1 knife, 424.0ms
Speed: 23.1ms preprocess, 424.0ms inference, 14.9ms postprocess per image at shape (1, 3, 736, 928)
⚠️ Alert: knife detected! Confidence: 0.79
🔊 Calling play_alarm('knife')
🚨 Playing alarm for knife...
✅ Objects detected: knife


In [19]:
image_path = r"D:\Explosive.jpg"
detect_objects(image_path)



0: 544x928 1 Explosive_Objects, 426.2ms
Speed: 13.7ms preprocess, 426.2ms inference, 18.7ms postprocess per image at shape (1, 3, 544, 928)
⚠️ Alert: explosive_objects detected! Confidence: 0.92
🔊 Calling play_alarm('explosive_objects')
🚨 Playing alarm for explosive_objects...
✅ Objects detected: explosive_objects


In [20]:
image_path = r"D:\Gun.jpg"
detect_objects(image_path)



0: 800x928 1 Gun, 416.6ms
Speed: 25.1ms preprocess, 416.6ms inference, 1.6ms postprocess per image at shape (1, 3, 800, 928)
⚠️ Alert: gun detected! Confidence: 0.91
🔊 Calling play_alarm('gun')
🚨 Playing alarm for gun...
✅ Objects detected: gun
